In [ ]:
import numpy as np
import pandas as pd
import json
import sklearn
from sklearn.preprocessing import LabelEncoder 
from pandas.io.json import json_normalize
import seaborn as sns
sns.set_style("whitegrid")
from pandas.plotting import scatter_matrix

In [65]:
"""
Useful Sections:
Regression Notebook 3 "Why add Qualitative Data to Regression"
Regression Notebook 3 "How to Choose the Best Model" - k-fold cross validation

"""

'\nUseful Sections:\nRegression Notebook 3 \n'

In [54]:
bot_labels = pd.read_csv('data/botwiki-2019.tsv', names = ['id', 'identity'], sep = '\t')
human_labels = pd.read_csv('data/verified-2019.tsv', names = ['id', 'identity'], sep = '\t')
bot_labels['bot'] = 1
human_labels['bot'] = 0
#IMPROVE?
# First set all beers as 0
#beers_train['Stout'] = 0
# Then locate all the stouts and set Stout to 1
#beers_train.loc[beers_train.Beer_Type == "Stout",'Stout'] = 1

In [55]:
#if IOUP data rate exceeded: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

#convert json to DataFrame
file = open('data/botwiki-2019_tweets.json')
file1 = open('data/verified-2019_tweets.json')
jsn = json.load(file)
jsn1 = json.load(file1)
bot_accts = pd.DataFrame(pd.json_normalize(jsn))
human_accts = pd.DataFrame(pd.json_normalize(jsn1))

In [56]:
#print(bot_tweets)
bot_tweets.head()

,key_0,created_at,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,...,user.has_extended_profile,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.translator_type,id,identity,bot
0,969398693249126400,Wed Jun 05 21:43:20 +0000 2019,969398693249126400,969398693249126400,Random Goth Lyric Generator,goth_lyrics,City of Dis,Esoteric dark verse from aeons before twitter ...,https://t.co/s9UkVMDACm,"[{'url': 'https://t.co/s9UkVMDACm', 'expanded_...",...,True,False,False,False,False,False,none,969398693249126400,bot,1
1,2402022649,Tue Apr 02 13:26:02 +0000 2019,2402022649,2402022649,Transiting Now,transitingnow,Earth,Demonstrating the vast number of transiting ex...,https://t.co/L8ARV5JWBz,"[{'url': 'https://t.co/L8ARV5JWBz', 'expanded_...",...,False,False,False,False,False,False,none,2402022649,bot,1
2,2976541239,Wed Jun 05 20:22:03 +0000 2019,2976541239,2976541239,TheRiddlerBot,TheRiddlerBot,"Coimbra, Portugal",Solve the riddle by replying only the name of ...,https://t.co/1v8BON9QpT,"[{'url': 'https://t.co/1v8BON9QpT', 'expanded_...",...,False,True,False,False,False,False,none,2976541239,bot,1
3,4425828741,Wed Jun 05 18:56:21 +0000 2019,4425828741,4425828741,🌊🌊🌊🌊🌊🌊🌊🌊🌊,tiny_seas,sui.sexy,✨🌊 by @suisea ✨ donate: https://t.co/MS5qM3a8B...,https://t.co/Q6hC8oOXfQ,"[{'url': 'https://t.co/Q6hC8oOXfQ', 'expanded_...",...,False,True,False,False,False,False,none,4425828741,bot,1
4,817082622824169472,Wed Jun 05 20:54:02 +0000 2019,817082622824169472,817082622824169472,Wordfindatron,wordfindatron,The Grid,I make little word-find puzzles. Bot by @joshl...,https://t.co/Dn62jWBqAe,"[{'url': 'https://t.co/Dn62jWBqAe', 'expanded_...",...,False,False,False,False,False,False,none,817082622824169472,bot,1


In [57]:
#merge users and labels
bot_accts = bot_accts.merge(bot_labels, left_on=bot_tweets['user.id'], right_on=bot_labels['id'])
human_accts = human_accts.merge(human_labels, left_on=human_accts['user.id'], right_on=human_labels['id'])
accts = bot_accts.append(human_accts)

In [58]:
tweets.tail()

,key_0,created_at,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,...,user.has_extended_profile,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.translator_type,id,identity,bot
1982,76235237,Wed Jun 05 12:26:47 +0000 2019,76235237,76235237,Tire Discounters,TireDiscounters,Higher Standards.,Family owned & operated business for 40 years!...,https://t.co/XWFZ4aqcZ4,"[{'url': 'https://t.co/XWFZ4aqcZ4', 'expanded_...",...,True,False,False,False,False,False,none,76235237,human,0
1983,2159329092,Wed Jun 05 22:07:56 +0000 2019,2159329092,2159329092,eillie anzilotti,eillieanzi,"Seattle, WA","(it's eye-lee.) sustainability, cities, and eq...",https://t.co/OJ69l0pxyk,"[{'url': 'https://t.co/OJ69l0pxyk', 'expanded_...",...,False,False,False,False,False,False,none,2159329092,human,0
1984,25527618,Wed Jun 05 14:16:01 +0000 2019,25527618,25527618,Debra Birnbaum,debrabirnbaum,Los Angeles,Head of awards at Amazon. Former editor at Var...,https://t.co/IiGhu084D5,"[{'url': 'https://t.co/IiGhu084D5', 'expanded_...",...,True,False,False,False,False,False,none,25527618,human,0
1985,43654274,Wed Jun 05 16:33:33 +0000 2019,43654274,43654274,艾未未 Ai Weiwei,aiww,北京,Instagram: https://t.co/dASr1tt9PE https://t.c...,https://t.co/wBpiJ5kRRf,"[{'url': 'https://t.co/wBpiJ5kRRf', 'expanded_...",...,False,False,False,False,False,False,none,43654274,human,0
1986,19084210,Thu Jun 06 01:00:12 +0000 2019,19084210,19084210,LexisNexis,LexisNexis,"New York, NY","The official feed of LexisNexis®, a leading gl...",None,NaN,...,False,False,False,False,False,False,none,19084210,human,0


In [59]:
#Test Train Split
accts_copy = accts.copy()
accts_train = accts_copy.sample(frac = 0.75, random_state = 1400)
accts_test = accts_copy.drop(accts_train.index)

In [66]:
#K-fold cross validation
from sklearn.model_selection import KFold
from sklearn.base import clone